In [1]:
seed = 2023
from nlpsig_networks.scripts.swnu_network_functions import (
    obtain_SWNUNetwork_input
)
from nlpsig_networks.utils import obtain_paths_mask, obtain_signatures_mask

In [2]:
import os
os.chdir("Rumours/")

## Rumours

In [3]:
%run load_sbert-embeddings.py

In [4]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [5]:
len(df_rumours)

5568

In [6]:
x_data = obtain_SWNUNetwork_input(
    method="umap",
    dimension=3,
    df=df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings=sbert_embeddings,
    k=10,
    features=['time_encoding', 'timeline_index'],
    standardise_method=None,
    include_features_in_path=False,
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [7]:
x_data.keys()

dict_keys(['x_data', 'input_channels', 'embedding_dim', 'num_features'])

In [8]:
x_data["x_data"]

{'path': tensor([[[ 4.2708, 11.1495, 10.6158],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[ 4.2708, 11.1495, 10.6158],
          [ 3.4460,  9.6966, 12.5073],
          [ 0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[ 4.2708, 11.1495, 10.6158],
          [ 3.4460,  9.6966, 12.5073],
          [ 9.4265,  7.5356, 12.5888],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         ...,
 
         [[ 4.9594, 10.2321, 11.3985],
          [ 0.8624,  9.3122, 11.9263],
          [ 2.2617, 10.3596, 12.9991],
          ...,
          [ 2.4673,  7.1607, 12.4286],
          [ 2.9867,  7.8778, 12.6077],
          

In [9]:
x_data["x_data"]["path"].shape

torch.Size([5568, 10, 3])

In [10]:
x_data["x_data"]["features"]

tensor([[ 6.4823e-02,  9.3748e-03, -3.3364e-02,  ...,  1.8542e-03,
          2.0148e+03,  1.0000e+00],
        [-2.9260e-02, -5.3905e-02, -3.9863e-02,  ...,  9.3606e-02,
          2.0148e+03,  2.0000e+00],
        [-3.1863e-02,  5.5065e-02, -2.4028e-02,  ...,  9.3310e-02,
          2.0148e+03,  3.0000e+00],
        ...,
        [-2.2057e-03,  8.0860e-02, -9.9185e-02,  ..., -2.3929e-02,
          2.0148e+03,  2.2000e+01],
        [-3.1252e-02,  3.5479e-02,  7.2934e-03,  ..., -1.3536e-02,
          2.0148e+03,  2.3000e+01],
        [ 1.1187e-02,  8.0548e-02, -4.1098e-02,  ...,  8.9290e-02,
          2.0148e+03,  2.4000e+01]])

In [11]:
x_data["input_channels"]

3

In [12]:
x_data["embedding_dim"]

384

In [13]:
x_data["num_features"]

2

In [14]:
path = x_data["x_data"]["path"]

In [15]:
path.shape

torch.Size([5568, 10, 3])

In [16]:
import signatory

In [17]:
signatures = signatory.logsignature(path, 2, stream=True)

In [18]:
signatures.shape

torch.Size([5568, 9, 6])

In [19]:
path[0]

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [20]:
signatures[0]

tensor([[ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000]],
       dtype=torch.float64)

In [21]:
path[2]

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 9.4265,  7.5356, 12.5888],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [22]:
signatures[2]

tensor([[ -0.8247,  -1.4529,   1.8914,   0.0000,   0.0000,   0.0000],
        [  5.1557,  -3.6139,   1.9729,   5.2355,  -5.6894,   1.9845],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654]],
       dtype=torch.float64)

`True` values in the mask are the ones that indicate to MultiheadAttention to ignore in attention calculation.

In [23]:
path_mask = obtain_paths_mask(path)
signatures_mask = obtain_signatures_mask(signatures)

In [24]:
path_mask.shape

torch.Size([5568, 10])

In [25]:
signatures_mask.shape

torch.Size([5568, 9])

In [26]:
path_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [27]:
signatures_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [28]:
path_mask_ = (torch.sum(path, 2) == 0)

In [29]:
signatures_mask_ = path_mask[:,:-1]

In [30]:
path_mask_[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [31]:
torch.equal(path_mask, path_mask_)

True

In [32]:
torch.equal(signatures_mask, signatures_mask_)

True

In [33]:
path_mask.dtype

torch.bool

In [34]:
path[0]

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [35]:
signatures[0]

tensor([[ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000]],
       dtype=torch.float64)

In [36]:
path_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [37]:
signatures_mask[0]

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True])

In [38]:
path[1]

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)

In [39]:
signatures[1]

tensor([[ -0.8247,  -1.4529,   1.8914,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558]],
       dtype=torch.float64)

In [40]:
path_mask[1]

tensor([False, False,  True,  True,  True,  True,  True,  True,  True,  True])

In [41]:
signatures_mask[1]

tensor([False, False,  True,  True,  True,  True,  True,  True,  True])

## Path mask

In [42]:
import torch.nn as nn
mha = nn.MultiheadAttention(
    embed_dim=path.shape[2],
    num_heads=1,
    batch_first=True,
).double()

In [43]:
attn_output, attn_output_weights = mha(path.double(),
                                       path.double(),
                                       path.double(),
                                       key_padding_mask=path_mask)

In [44]:
i=0
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155],
        [ 0.2119, -3.1534, -2.9155]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [45]:
i=1
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False,  True,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 0.1054, -2.6965, -2.4112],
        [ 0.1054, -2.6964, -2.4110],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631],
        [ 0.1586, -2.9248, -2.6631]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [46]:
i=2
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 9.4265,  7.5356, 12.5888],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False, False,  True,  True,  True,  True,  True,  True,  True])
tensor([[ 0.1054, -2.6965, -2.4112],
        [ 0.1054, -2.6964, -2.4110],
        [ 0.1065, -2.7013, -2.4164],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113],
        [ 0.3979, -3.3791, -2.5113]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


In [47]:
i=4
print(path[i])
print(path_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 9.4265,  7.5356, 12.5888],
        [ 4.4903, 11.1706, 10.8094],
        [ 4.4021, 10.7961, 13.9463],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([False, False, False, False, False,  True,  True,  True,  True,  True])
tensor([[ 0.1055, -2.6969, -2.4114],
        [ 0.1055, -2.6967, -2.4112],
        [ 0.1073, -2.7047, -2.4193],
        [ 0.1055, -2.6969, -2.4114],
        [ 0.1054, -2.6965, -2.4110],
        [ 0.3233, -3.3099, -2.6364],
        [ 0.3233, -3.3099, -2.6364],
        [ 0.3233, -3.3099, -2.6364],
        [ 0.3233, -3.3099, -2.6364],
        [ 0.3233, -3.3099, -2.6364]], dtype=torch.float64,
       grad_fn=<SelectBackward>)


## Signatures mask

In [48]:
import torch.nn as nn
mha = nn.MultiheadAttention(
    embed_dim=signatures.shape[2],
    num_heads=1,
    batch_first=True,
).double()

In [49]:
attn_output, attn_output_weights = mha(signatures.double(),
                                       signatures.double(),
                                       signatures.double(),
                                       key_padding_mask=signatures_mask)

In [50]:
i=0
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   0.0000,

In [51]:
i=1
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ -0.8247,  -1.4529,   1.8914,   0.0000,   0.0000,   0.0000],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,   8.4165,  18.2558],
        [ -4.2708, -11.1495, -10.6158,   1.4952,

In [52]:
i=2
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 9.4265,  7.5356, 12.5888],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[ -0.8247,  -1.4529,   1.8914,   0.0000,   0.0000,   0.0000],
        [  5.1557,  -3.6139,   1.9729,   5.2355,  -5.6894,   1.9845],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234, -28.8427,  32.1654],
        [ -4.2708, -11.1495, -10.6158, -31.2234,

In [53]:
i=4
print(path[i])
print(signatures[i])
print(signatures_mask[i])
print(attn_output[i])

tensor([[ 4.2708, 11.1495, 10.6158],
        [ 3.4460,  9.6966, 12.5073],
        [ 9.4265,  7.5356, 12.5888],
        [ 4.4903, 11.1706, 10.8094],
        [ 4.4021, 10.7961, 13.9463],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], dtype=torch.float64)
tensor([[-8.2473e-01, -1.4529e+00,  1.8914e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 5.1557e+00, -3.6139e+00,  1.9729e+00,  5.2355e+00, -5.6894e+00,
          1.9845e+00],
        [ 2.1951e-01,  2.1107e-02,  1.9351e-01,  5.6866e+00, -5.4071e+00,
          1.6140e+00],
        [ 1.3132e-01, -3.5339e-01,  3.3305e+00,  5.6464e+00, -5.0542e+00,
          1.6833e+00],
        [-4.2708e+00, -1.1149e+01, -1.0616e+01,  4.1597e+00,  1.3605e+00,
          2.2126e+01],
        [-4.2708e+00, -1.1149e+01, -1.0616e+01,  4.1597e+00,  1.3605e+00,
          2.2126e+01],
        [-4.2708e+00, -1.1